In [1]:
import os

os.chdir("..")

from utils import data as data_utils
import torch_geometric as pyg
from utils import ZincWithRDKit
from datasets import DatasetDict, load_from_disk, IterableDataset
from os.path import join
import subprocess
from utils import graphormer_data_collator_improved as graphormer_collator_utils
from tqdm import tqdm
import numpy as np
import time


data_dir = "./data"

/home/alexander/miniconda3/envs/pgt_hug/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/pcqm4mv2/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [3]:
dataset[0]

{'edge_attr': [[12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [1, 0, 0],
  [1, 0, 0],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [1, 0, 0],
  [1, 0, 0],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 0],
  [1, 0, 1],
  [1, 0, 1],
  [2, 0, 1],
  [2, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [12, 0, 1],
  [1, 0, 1],
  [1, 0, 1]],
 'node_feat': [[6, 0, 4, 5, 3, 0, 4, 0, 0],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 1, 1, 4, 0, 1],
  [6, 0, 3, 5, 1, 0, 3, 1, 1],
  [6, 0, 3, 5, 0, 0, 3, 1, 1],
  [6, 0, 3, 5, 0, 0, 3, 1, 1],
  [6, 0, 3, 5, 0, 0, 3, 1, 1],
  [6, 1, 4, 5, 1, 0, 4, 0, 1],
  [6, 0, 3, 5, 0, 0, 3, 1,

In [4]:
dataset[0]["node_feat"]

[[6, 0, 4, 5, 3, 0, 4, 0, 0],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 1, 4, 0, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 1, 4, 5, 1, 0, 4, 0, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 0, 1],
 [7, 0, 2, 5, 0, 0, 3, 1, 1],
 [7, 0, 2, 5, 0, 1, 3, 0, 1],
 [8, 0, 1, 5, 0, 0, 3, 0, 0]]

In [5]:
max(dataset[0]["node_feat"])

[8, 0, 1, 5, 0, 0, 3, 0, 0]

In [6]:
max(max(dataset[0]["node_feat"]))

8

In [7]:
true_max = 0
for i in tqdm(dataset_size):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

0it [00:00, ?it/s]


TypeError: 'int' object is not iterable

In [8]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  0%|          | 503/3378606 [00:00<21:16, 2646.58it/s]

8 0
9 3
17 4
35 99


100%|██████████| 3378606/3378606 [17:58<00:00, 3134.02it/s]


In [9]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/tox21_original/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [10]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

 20%|██        | 131/645 [00:00<00:01, 313.89it/s]

8 0
35 1
53 24


100%|██████████| 645/645 [00:01<00:00, 627.60it/s]

80 498


In [11]:
dataset_size

645

In [12]:
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)

In [13]:
dataset

Dataset({
    features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
    num_rows: 645
})

In [14]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/tox21_original/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [15]:
dataset

Dataset({
    features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
    num_rows: 11758
})

In [16]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  2%|▏         | 230/11758 [00:00<00:08, 1315.59it/s]

17 0
35 1
80 6


  7%|▋         | 870/11758 [00:00<00:05, 2012.81it/s]

83 617


100%|██████████| 11758/11758 [00:04<00:00, 2556.41it/s]


In [17]:
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)


In [18]:
dataset

DatasetDict({
    train: Dataset({
        features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
        num_rows: 11758
    })
    validation: Dataset({
        features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
        num_rows: 295
    })
    test: Dataset({
        features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
        num_rows: 645
    })
})

In [19]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/tox21_original/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["validation"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [20]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

100%|██████████| 295/295 [00:00<00:00, 2142.76it/s]

8 0
17 2
35 3
44 13
53 29


In [21]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/tox21/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["validation"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [22]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  6%|▌         | 445/7555 [00:00<00:02, 2373.14it/s]

16 0
17 6
53 7
80 240


 16%|█▋        | 1231/7555 [00:00<00:02, 2117.74it/s]

81 951


 32%|███▏      | 2394/7555 [00:00<00:01, 2656.57it/s]

82 1870


 40%|███▉      | 3001/7555 [00:01<00:01, 2639.17it/s]

83 2630


100%|██████████| 7555/7555 [00:02<00:00, 2729.28it/s]


In [23]:
dataset[0]["node_feat"]

[[6, 0, 4, 5, 3, 0, 4, 0, 0],
 [6, 0, 4, 5, 2, 0, 4, 0, 0],
 [8, 0, 2, 5, 0, 0, 3, 0, 0],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [7, 0, 2, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [16, 0, 4, 5, 0, 0, 4, 0, 0],
 [7, 0, 3, 5, 2, 0, 4, 0, 0],
 [8, 0, 1, 5, 0, 0, 3, 0, 0],
 [8, 0, 1, 5, 0, 0, 3, 0, 0],
 [16, 0, 2, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 0, 0, 3, 1, 1],
 [6, 0, 3, 5, 1, 0, 3, 1, 1]]

In [24]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/ZINC/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["validation"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [25]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

 31%|███       | 310/1000 [00:00<00:00, 1686.67it/s]

16 0
17 2
53 36


100%|██████████| 1000/1000 [00:00<00:00, 2129.74it/s]


In [26]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/ZINC/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["test"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [27]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

 78%|███████▊  | 785/1000 [00:00<00:00, 1491.85it/s]

17 0
53 6


100%|██████████| 1000/1000 [00:00<00:00, 1140.04it/s]



In [28]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/ZINC/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = DatasetDict.load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [29]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  4%|▍         | 403/10000 [00:00<00:11, 820.56it/s]

16 0
17 2
35 24
53 123


100%|██████████| 10000/10000 [00:04<00:00, 2130.08it/s]


In [30]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/qm9/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [31]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  1%|          | 976/133247 [00:00<00:26, 4990.71it/s]

6 0
7 1
8 2
9 183


100%|██████████| 133247/133247 [00:33<00:00, 4018.66it/s]


In [32]:
dataset

Dataset({
    features: ['edge_attr', 'node_feat', 'smiles', 'num_nodes', 'pos', 'edge_index', 'name', 'id', 'target'],
    num_rows: 133247
})

In [33]:
dataset[0]["smiles"]

'C'

In [34]:
dataset[12312]["smiles"]

'CC1C=CCCC1C'

In [35]:
#collator = graphormer_collator_utils.GraphormerDataCollator()
#dataset_path =  "/home/alexander/temp/ZINC/processed/arrow"
dataset_path =  "data/pcba/processed/arrow"
#dataset_path =  "data/ZINC/processed/arrow_processed"
dataset = load_from_disk(
    dataset_path, keep_in_memory=False
)
if isinstance(dataset, DatasetDict):
    dataset = dataset["train"]


dataset.cleanup_cache_files()
dataset_size = len(dataset)
tot_batches = 1000
batch_size = 256

#dataset = dataset.shuffle()

In [36]:
true_max = 0
for i in tqdm(range(dataset_size)):
    sample_max = max(max(dataset[i]["node_feat"]))
    if sample_max > true_max:
        true_max = sample_max
        print(true_max, i)
    #samples.append(dataset[i + 1000000])

  0%|          | 439/437929 [00:00<03:12, 2272.76it/s]

8 0
17 1
35 13
53 565


  1%|▏         | 6280/437929 [00:02<02:53, 2481.28it/s]

78 6005


  3%|▎         | 13534/437929 [00:05<02:50, 2487.88it/s]

80 13059


  8%|▊         | 35197/437929 [00:14<02:53, 2321.17it/s]

83 34821


100%|██████████| 437929/437929 [03:15<00:00, 2240.69it/s]
